# Рубежный контроль №2

## Юсипов Артур ИУ5-24М
## Тема: Методы обработки текстов
## Решение задачи классификации текстов.

- Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на
основе TfidfVectorizer.

- В качестве классификаторов необходимо использовать два классификатора по варианту для Вашей
группы
### Группа: ИУ5-24М
### Классификатор 1: KNeighborsClassifier
### Классификатор 2: Complement Naive Bayes (CNB)

- Для каждого метода необходимо оценить качество классификации
- Сделать вывод о том, какой вариант векторизации признаков в паре с каким классификатором показал лучшее качество.

<!-- Выбранный [Датасет]() -->

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install category_encoders

You should consider upgrading via the 'c:\users\arthur\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install kaggle
import os
os.environ['KAGGLE_USERNAME'] = "XXX" 
os.environ['KAGGLE_KEY'] = "XXX"
!kaggle datasets download clmentbisaillon/fake-and-real-news-dataset
!unzip fake-and-real-news-dataset.zip

You should consider upgrading via the 'c:\users\arthur\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.



  0%|          | 0.00/41.0M [00:00<?, ?B/s]
  2%|▏         | 1.00M/41.0M [00:02<01:51, 375kB/s]
  5%|▍         | 2.00M/41.0M [00:04<01:28, 464kB/s]
  7%|▋         | 3.00M/41.0M [00:06<01:17, 511kB/s]
 10%|▉         | 4.00M/41.0M [00:07<01:04, 600kB/s]
 12%|█▏        | 5.00M/41.0M [00:09<00:57, 657kB/s]
 15%|█▍        | 6.00M/41.0M [00:10<00:57, 642kB/s]
 17%|█▋        | 7.00M/41.0M [00:12<00:53, 667kB/s]
 20%|█▉        | 8.00M/41.0M [00:13<00:52, 653kB/s]
 22%|██▏       | 9.00M/41.0M [00:15<00:53, 622kB/s]
 24%|██▍       | 10.0M/41.0M [00:17<00:51, 631kB/s]
 27%|██▋       | 11.0M/41.0M [00:19<00:52, 597kB/s]
 29%|██▉       | 12.0M/41.0M [00:21<00:50, 606kB/s]
 32%|███▏      | 13.0M/41.0M [00:22<00:48, 607kB/s]
 34%|███▍      | 14.0M/41.0M [00:24<00:49, 572kB/s]
 37%|███▋      | 15.0M/41.0M [00:27<00:53, 506kB/s]
 39%|███▉      | 16.0M/41.0M [00:28<00:45, 571kB/s]
 41%|████▏     | 17.0M/41.0M [00:29<00:37, 664kB/s]
 44%|████▍     | 18.0M/41.0M [00:30<00:32, 736kB/s]
 46%|████▋     | 19

## Анализируем датасет и готовим категориальный признак

In [6]:
SUBSAMPLE_SIZE = 10000

df_fake = pd.read_csv('Fake.csv', encoding='utf-8')[:SUBSAMPLE_SIZE]
df_fake['target'] = np.zeros(df_fake.shape[0], dtype=np.int8)
df_true = pd.read_csv('True.csv', encoding='utf-8')[:SUBSAMPLE_SIZE]
df_true['target'] = np.ones(df_true.shape[0], dtype=np.int8)
df = pd.concat((df_fake, df_true), axis=0)
df.sample(frac=1).reset_index(drop=True)
df

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
9995,Obama says Clinton never jeopardized national ...,WASHINGTON (Reuters) - U.S. President Barack O...,politicsNews,"April 10, 2016",1
9996,U.S. plans to curb tax 'inversions' could hit ...,LONDON (Reuters) - Planned changes that Presid...,politicsNews,"April 11, 2016",1
9997,U.S. Democrat Clinton downplays chance of cont...,WASHINGTON (Reuters) - Democratic front-runner...,politicsNews,"April 10, 2016",1
9998,Boston Globe denounces Trump candidacy in 'fro...,(Reuters) - Headlines screaming “Deportations ...,politicsNews,"April 10, 2016",1


In [7]:
df.target.value_counts()

1    10000
0    10000
Name: target, dtype: int64

In [8]:
df.subject.value_counts()

politicsNews    10000
News             9050
politics          950
Name: subject, dtype: int64

In [9]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder

encoder = LabelEncoder()
scaler = StandardScaler()
df['subject'] = encoder.fit_transform(df.subject)

In [10]:
df.subject.unique()

array([0, 1, 2])

In [11]:
(df.subject - df.target).sum()

10950

In [12]:
df.date.value_counts()

November 9, 2016      115
April 7, 2017          73
February 1, 2017       51
February 2, 2017       49
January 23, 2017       47
                     ... 
May 14, 2016            1
December 11, 2017       1
October 14, 2017        1
December 30, 2017       1
August 21, 2016         1
Name: date, Length: 1480, dtype: int64

In [13]:
df.drop(columns=['date'], inplace=True)

## Сразу делим данные на две выборки train и test

In [14]:
from sklearn.model_selection import train_test_split

X = df[[i for i in df.columns if i !='target']]
y = df.target

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((16000, 3), (16000,), (4000, 3), (4000,))

## Предобрабатываем текстовые данные

In [15]:
for title in df.title:
  print(title, type(title))
  break

 Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing <class 'str'>


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re

def preproc_func(title):
  # make more preprocessing if it well be needed!
  if isinstance(title, str):
      title = re.sub('[^a-zA-Z0-9]', ' ', title)
      return title.lower().strip()
  else:
    return ''

def get_tfidf_matrix(df, column, preproc_function, vectorizer=None):
  """
    returns matrix, trained vectorizer
  """
  processed_col = df[column].apply(preproc_function)
  if vectorizer is None:
    vectorizer = TfidfVectorizer()
    vectorizer.fit(processed_col)

  matrix = vectorizer.transform(processed_col)
  
  return matrix, vectorizer

def get_count_matrix(df, column, preproc_function, vectorizer=None):
  """
    returns matrix, trained vectorizer
  """
  processed_col = df[column].apply(preproc_function)
  if vectorizer is None:
    vectorizer = CountVectorizer()
    vectorizer.fit(processed_col)

  matrix = vectorizer.transform(processed_col)
  return matrix, vectorizer


train_title_matrix_tfidf, tfidf_vectorizer = get_tfidf_matrix(x_train, 'title', preproc_func)
test_title_matrix_tfidf, tfidf_vectorizer = get_tfidf_matrix(x_test, 'title',
                                                       preproc_func, vectorizer=tfidf_vectorizer)

train_title_matrix_count, count_vectorizer = get_count_matrix(x_train, 'title', preproc_func)
test_title_matrix_count, count_vectorizer = get_count_matrix(x_test, 'title',
                                                       preproc_func, vectorizer=count_vectorizer)

In [17]:
train_title_matrix_tfidf.shape, test_title_matrix_tfidf.shape, train_title_matrix_count.shape, test_title_matrix_count.shape

((16000, 13240), (4000, 13240), (16000, 13240), (4000, 13240))

In [18]:
train_text_matrix_tfidf, tfidf_text_vectorizer = get_tfidf_matrix(x_train, 'text', preproc_func)
test_text_matrix_tfidf, tfidf_text_vectorizer = get_tfidf_matrix(x_test, 'text',
                                                       preproc_func, vectorizer=tfidf_text_vectorizer)

train_text_matrix_count, count_text_vectorizer = get_tfidf_matrix(x_train, 'text', preproc_func)
test_text_matrix_count, count_text_vectorizer = get_tfidf_matrix(x_test, 'text',
                                                       preproc_func, vectorizer=count_text_vectorizer)

In [19]:
train_text_matrix_tfidf.shape, test_text_matrix_tfidf.shape

((16000, 73574), (4000, 73574))

In [20]:
train_text_matrix_count.shape, test_text_matrix_count.shape

((16000, 73574), (4000, 73574))

In [21]:
from scipy import sparse

subject_train_sparse = sparse.csr_matrix(np.array(x_train.subject).reshape(-1, 1))
subject_test_sparse = sparse.csr_matrix(np.array(x_test.subject).reshape(-1, 1))

# subject_train_sparse.shape
texts_tfidf_train_matrix = sparse.hstack((train_text_matrix_tfidf,train_title_matrix_tfidf, subject_train_sparse))
texts_count_train_matrix = sparse.hstack((train_text_matrix_count,train_title_matrix_count, subject_train_sparse))
texts_tfidf_test_matrix = sparse.hstack((test_text_matrix_tfidf,test_title_matrix_tfidf, subject_test_sparse))
texts_count_test_matrix = sparse.hstack((test_text_matrix_count,test_title_matrix_count, subject_test_sparse))

In [22]:
texts_tfidf_train_matrix.shape

(16000, 86815)

## KNN with CountVectorizer

In [23]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


parameters = {'n_neighbors': [2, 3, 5, 7, 9]}


knn_clf = KNeighborsClassifier()

knn_grid_count_clf = GridSearchCV(knn_clf, parameters, verbose=4, scoring='f1_macro', n_jobs=-1, cv=3)

knn_grid_count_clf.fit(texts_count_train_matrix, y_train)

pd.DataFrame(knn_grid_count_clf.cv_results_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.068665,0.001695,14.359868,0.047684,2,{'n_neighbors': 2},0.980308,0.975611,0.978240,0.978053,0.001922,1
1,0.066993,0.002455,14.629618,0.153571,3,{'n_neighbors': 3},0.970165,0.963009,0.967715,0.966963,0.002969,2
2,0.070530,0.005369,15.269342,0.605137,5,{'n_neighbors': 5},0.968849,0.960184,0.965457,0.964830,0.003565,3
3,0.066669,0.002608,16.109434,0.048507,7,{'n_neighbors': 7},0.968473,0.959619,0.962445,0.963512,0.003692,4
4,0.064319,0.001900,12.447080,0.069296,9,{'n_neighbors': 9},0.967532,0.957922,0.961880,0.962445,0.003944,5


In [25]:
best_knn_count_clf = KNeighborsClassifier(n_neighbors=2)
best_knn_count_clf.fit(texts_count_train_matrix, y_train)

pred = best_knn_count_clf.predict(X=texts_count_test_matrix)

best_knn_count = classification_report(y_test, pred, digits=4, output_dict=True)

print(classification_report(y_test, pred, digits=4) )

              precision    recall  f1-score   support

           0     0.9979    0.9645    0.9809      2000
           1     0.9657    0.9980    0.9816      2000

    accuracy                         0.9812      4000
   macro avg     0.9818    0.9812    0.9812      4000
weighted avg     0.9818    0.9812    0.9812      4000



## KNN with TfidfVectorizer

In [26]:
parameters = {'n_neighbors': [2, 3, 5, 7, 9]}


knn_clf = KNeighborsClassifier()

knn_grid_tfidf_clf = GridSearchCV(knn_clf, parameters, verbose=4, scoring='f1_macro', n_jobs=-1, cv=3)

knn_grid_tfidf_clf.fit(texts_tfidf_train_matrix, y_train)

pd.DataFrame(knn_grid_tfidf_clf.cv_results_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.074137,0.003671,14.531715,0.077497,2,{'n_neighbors': 2},0.999438,1.0,0.999625,0.999688,0.000234,4
1,0.068825,0.006023,14.795847,0.137342,3,{'n_neighbors': 3},0.999438,1.0,0.999437,0.999625,0.000265,5
2,0.067547,0.006255,15.125217,0.302835,5,{'n_neighbors': 5},0.999438,1.0,0.999812,0.999750,0.000234,3
3,0.065339,0.004700,15.211160,0.124978,7,{'n_neighbors': 7},0.999813,1.0,0.999812,0.999875,0.000088,2
4,0.071679,0.006791,12.580699,0.063409,9,{'n_neighbors': 9},1.000000,1.0,0.999812,0.999937,0.000088,1


In [27]:
best_knn_clf_tfidf = KNeighborsClassifier(n_neighbors=9)

best_knn_clf_tfidf.fit(texts_tfidf_train_matrix, y_train)

pred_tfidf = best_knn_clf_tfidf.predict(X=texts_tfidf_test_matrix)

best_knn_tfidf = classification_report(y_test, pred_tfidf, digits=4, output_dict=True)

print(classification_report(y_test, pred_tfidf, digits=4))

              precision    recall  f1-score   support

           0     1.0000    0.9995    0.9997      2000
           1     0.9995    1.0000    0.9998      2000

    accuracy                         0.9998      4000
   macro avg     0.9998    0.9998    0.9997      4000
weighted avg     0.9998    0.9998    0.9997      4000



## Complement Bayes with CountVectorizer

In [28]:
from sklearn.naive_bayes import ComplementNB

parameters = {'alpha': [0, 0.5, 1, 2, 4], 'norm': [True, False]}

comp_clf = ComplementNB()

comp_grid_count_clf = GridSearchCV(comp_clf, parameters, verbose=4, scoring='f1_macro', n_jobs=-1, cv=3)

comp_grid_count_clf.fit(texts_count_train_matrix, y_train)

pd.DataFrame(comp_grid_count_clf.cv_results_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.112033,0.000470,0.025598,0.000470,0,True,"{'alpha': 0, 'norm': True}",0.812293,0.783548,0.806871,0.800904,0.012471,10
1,0.108708,0.011828,0.024934,0.002153,0,False,"{'alpha': 0, 'norm': False}",0.928691,0.930947,0.926784,0.928808,0.001702,9
2,0.107381,0.010182,0.029587,0.006632,0.5,True,"{'alpha': 0.5, 'norm': True}",0.980127,0.978808,0.980686,0.979873,0.000787,8
3,0.084441,0.016454,0.026263,0.001695,0.5,False,"{'alpha': 0.5, 'norm': False}",0.979939,0.979184,0.980686,0.979936,0.000613,7
4,0.087101,0.012252,0.025930,0.000815,1,True,"{'alpha': 1, 'norm': True}",0.981252,0.981810,0.982936,0.981999,0.000700,5
5,0.077867,0.014105,0.025323,0.000431,1,False,"{'alpha': 1, 'norm': False}",0.981439,0.980497,0.981811,0.981249,0.000553,6
6,0.089918,0.013433,0.026681,0.001684,2,True,"{'alpha': 2, 'norm': True}",0.983314,0.983310,0.984061,0.983562,0.000353,3
7,0.079303,0.013189,0.025336,0.000498,2,False,"{'alpha': 2, 'norm': False}",0.982939,0.982560,0.983311,0.982937,0.000307,4
8,0.098666,0.004919,0.024669,0.000460,4,True,"{'alpha': 4, 'norm': True}",0.984064,0.984811,0.985749,0.984874,0.000689,1
9,0.090268,0.006678,0.022824,0.005350,4,False,"{'alpha': 4, 'norm': False}",0.984439,0.983497,0.985186,0.984374,0.000691,2


In [29]:
comp_clf = ComplementNB(alpha=4, norm=True)

comp_clf.fit(texts_count_train_matrix, y_train)
bayes_pred = comp_clf.predict(X=texts_count_test_matrix)

best_comp_count = classification_report(y_test, bayes_pred, digits=4, output_dict=True)

print(classification_report(y_test, bayes_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9899    0.9810    0.9854      2000
           1     0.9812    0.9900    0.9856      2000

    accuracy                         0.9855      4000
   macro avg     0.9855    0.9855    0.9855      4000
weighted avg     0.9855    0.9855    0.9855      4000



## Complement Bayes with TfidfVectorizer

In [30]:
parameters = {'alpha': [0, 0.5, 1, 2, 4], 'norm': [True, False]}

comp_clf = ComplementNB()

comp_grid_tfidf_clf = GridSearchCV(comp_clf, parameters, verbose=4, scoring='f1_macro', n_jobs=-1, cv=3)

comp_grid_tfidf_clf.fit(texts_tfidf_train_matrix, y_train)

pd.DataFrame(comp_grid_tfidf_clf.cv_results_)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.102393,0.002352,0.024270,0.000470,0,True,"{'alpha': 0, 'norm': True}",0.703004,0.674528,0.706436,0.694656,0.014301,10
1,0.103721,0.003549,0.024269,0.001244,0,False,"{'alpha': 0, 'norm': False}",0.946349,0.949166,0.946147,0.947221,0.001378,9
2,0.101728,0.006361,0.023604,0.000470,0.5,True,"{'alpha': 0.5, 'norm': True}",0.973736,0.969031,0.976737,0.973168,0.003172,5
3,0.080782,0.012157,0.023936,0.000814,0.5,False,"{'alpha': 0.5, 'norm': False}",0.984249,0.980680,0.985184,0.983371,0.001941,1
4,0.088429,0.011781,0.023938,0.000814,1,True,"{'alpha': 1, 'norm': True}",0.974487,0.970159,0.977112,0.973919,0.002867,4
5,0.084773,0.019765,0.023271,0.000469,1,False,"{'alpha': 1, 'norm': False}",0.982935,0.978427,0.983683,0.981681,0.002322,2
6,0.091422,0.012987,0.024934,0.000815,2,True,"{'alpha': 2, 'norm': True}",0.973172,0.967338,0.975234,0.971914,0.003344,7
7,0.079788,0.012160,0.025597,0.001244,2,False,"{'alpha': 2, 'norm': False}",0.979557,0.975045,0.982369,0.978990,0.003017,3
8,0.100730,0.004953,0.024269,0.000471,4,True,"{'alpha': 4, 'norm': True}",0.968285,0.964327,0.971663,0.968092,0.002998,8
9,0.096745,0.003254,0.020608,0.001884,4,False,"{'alpha': 4, 'norm': False}",0.973547,0.968843,0.976736,0.973042,0.003242,6


In [31]:
comp_clf_tfidf = ComplementNB(alpha=0.5, norm=False)

comp_clf_tfidf.fit(texts_tfidf_train_matrix, y_train)
bayes_pred_tfidf = comp_clf_tfidf.predict(X=texts_tfidf_test_matrix)

best_comp_tfidf = classification_report(y_test, bayes_pred_tfidf, digits=4, output_dict=True)

print(classification_report(y_test, bayes_pred_tfidf, digits=4))

              precision    recall  f1-score   support

           0     0.9974    0.9735    0.9853      2000
           1     0.9741    0.9975    0.9857      2000

    accuracy                         0.9855      4000
   macro avg     0.9858    0.9855    0.9855      4000
weighted avg     0.9858    0.9855    0.9855      4000



## Итоговое способов

In [32]:
best_comp_count['macro avg']

models = ['ComplementNB CountVectorizer', 'ComplementNB TfidfVectorizer', 'KNeighborsClassifier CountVectorizer', 'KNeighborsClassifier TfidfVectorizer']
f1 = []
precision = []
recall = []

best_comp_count['macro avg']['f1-score']

for enum, i in enumerate([best_comp_count, best_comp_tfidf, best_knn_count, best_knn_tfidf]):
  # print(enum)
  f1.append(i['macro avg']['f1-score'])
  precision.append(i['macro avg']['precision'])
  recall.append(i['macro avg']['recall'])

pd.DataFrame({'labels':models, 'f1-score macro avg': f1, 'precision macro': precision, 'recall_macro': recall}).sort_values(by='f1-score macro avg', ascending=False).reset_index(drop=True)

,labels,f1-score macro avg,precision macro,recall_macro
0,KNeighborsClassifier TfidfVectorizer,0.999750,0.999750,0.99975
1,ComplementNB CountVectorizer,0.985500,0.985539,0.98550
2,ComplementNB TfidfVectorizer,0.985498,0.985780,0.98550
3,KNeighborsClassifier CountVectorizer,0.981245,0.981791,0.98125
